In [1]:
#this notebook looks as the race and income distribution of people that have access to citi bikes docks 
#within a 5-minute walk and compares that to the nyc race and income distribution
import pandas as pd
import geopandas as gpd
pd.options.display.max_columns = 10000
pd.options.display.max_rows = 10000

In [2]:
#import the 2016 american community survey and metadata 
acs_16 = pd.read_csv('~/Desktop/Pratt/fall2018/adv_gis/aff_download/ACS_16_5YR_S0601_with_ann.csv')
metadata_16 = pd.read_csv('~/Desktop/Pratt/fall2018/adv_gis/aff_download/ACS_16_5YR_S0601_metadata.csv')

/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,3,4,5,6,7,8,9,10,179,180,181,182,183,184,185,186,211,212,213,214,215,216,217,218,251,252,253,254,255,256,257,258,299,300,301,302,303,304,305,306,379,380,381,382,383,384,385,386) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
acs_16.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC04_EST_VC01,HC04_MOE_VC01,HC01_EST_VC03,HC01_MOE_VC03,HC02_EST_VC03,HC02_MOE_VC03,HC03_EST_VC03,HC03_MOE_VC03,HC04_EST_VC03,HC04_MOE_VC03,HC01_EST_VC04,HC01_MOE_VC04,HC02_EST_VC04,HC02_MOE_VC04,HC03_EST_VC04,HC03_MOE_VC04,HC04_EST_VC04,HC04_MOE_VC04,HC01_EST_VC05,HC01_MOE_VC05,HC02_EST_VC05,HC02_MOE_VC05,HC03_EST_VC05,HC03_MOE_VC05,HC04_EST_VC05,HC04_MOE_VC05,HC01_EST_VC06,HC01_MOE_VC06,HC02_EST_VC06,HC02_MOE_VC06,HC03_EST_VC06,HC03_MOE_VC06,HC04_EST_VC06,HC04_MOE_VC06,HC01_EST_VC07,HC01_MOE_VC07,HC02_EST_VC07,HC02_MOE_VC07,HC03_EST_VC07,HC03_MOE_VC07,HC04_EST_VC07,HC04_MOE_VC07,HC01_EST_VC08,HC01_MOE_VC08,HC02_EST_VC08,HC02_MOE_VC08,HC03_EST_VC08,HC03_MOE_VC08,HC04_EST_VC08,HC04_MOE_VC08,HC01_EST_VC09,HC01_MOE_VC09,HC02_EST_VC09,HC02_MOE_VC09,HC03_EST_VC09,HC03_MOE_VC09,HC04_EST_VC09,HC04_MOE_VC09,HC01_EST_VC10,HC01_MOE_VC10,HC02_EST_VC10,HC02_MOE_VC10,HC03_EST_VC10,HC03_MOE_VC10,HC04_EST_VC10,HC04_MOE_VC10,HC01_EST_VC12,HC01_MOE_VC12,HC02_EST_VC12,HC02_MOE_VC12,HC03_EST_VC12,HC03_MOE_VC12,HC04_EST_VC12,HC04_MOE_VC12,HC01_EST_VC15,HC01_MOE_VC15,HC02_EST_VC15,HC02_MOE_VC15,HC03_EST_VC15,HC03_MOE_VC15,HC04_EST_VC15,HC04_MOE_VC15,HC01_EST_VC16,HC01_MOE_VC16,HC02_EST_VC16,HC02_MOE_VC16,HC03_EST_VC16,HC03_MOE_VC16,HC04_EST_VC16,HC04_MOE_VC16,HC01_EST_VC19,HC01_MOE_VC19,HC02_EST_VC19,HC02_MOE_VC19,HC03_EST_VC19,HC03_MOE_VC19,HC04_EST_VC19,HC04_MOE_VC19,HC01_EST_VC20,HC01_MOE_VC20,HC02_EST_VC20,HC02_MOE_VC20,HC03_EST_VC20,HC03_MOE_VC20,HC04_EST_VC20,HC04_MOE_VC20,HC01_EST_VC21,HC01_MOE_VC21,HC02_EST_VC21,HC02_MOE_VC21,HC03_EST_VC21,HC03_MOE_VC21,HC04_EST_VC21,HC04_MOE_VC21,HC01_EST_VC22,HC01_MOE_VC22,HC02_EST_VC22,HC02_MOE_VC22,HC03_EST_VC22,HC03_MOE_VC22,HC04_EST_VC22,HC04_MOE_VC22,HC01_EST_VC23,HC01_MOE_VC23,HC02_EST_VC23,HC02_MOE_VC23,HC03_EST_VC23,HC03_MOE_VC23,HC04_EST_VC23,HC04_MOE_VC23,HC01_EST_VC24,HC01_MOE_VC24,HC02_EST_VC24,HC02_MOE_VC24,HC03_EST_VC24,HC03_MOE_VC24,HC04_EST_VC24,HC04_MOE_VC24,HC01_EST_VC25,HC01_MOE_VC25,HC02_EST_VC25,HC02_MOE_VC25,HC03_EST_VC25,HC03_MOE_VC25,HC04_EST_VC25,HC04_MOE_VC25,HC01_EST_VC26,HC01_MOE_VC26,HC02_EST_VC26,HC02_MOE_VC26,HC03_EST_VC26,HC03_MOE_VC26,HC04_EST_VC26,HC04_MOE_VC26,HC01_EST_VC28,HC01_MOE_VC28,HC02_EST_VC28,HC02_MOE_VC28,HC03_EST_VC28,HC03_MOE_VC28,HC04_EST_VC28,HC04_MOE_VC28,HC01_EST_VC29,HC01_MOE_VC29,HC02_EST_VC29,HC02_MOE_VC29,HC03_EST_VC29,HC03_MOE_VC29,HC04_EST_VC29,HC04_MOE_VC29,HC01_EST_VC32,HC01_MOE_VC32,HC02_EST_VC32,HC02_MOE_VC32,HC03_EST_VC32,HC03_MOE_VC32,HC04_EST_VC32,HC04_MOE_VC32,HC01_EST_VC33,HC01_MOE_VC33,HC02_EST_VC33,HC02_MOE_VC33,HC03_EST_VC33,HC03_MOE_VC33,HC04_EST_VC33,HC04_MOE_VC33,HC01_EST_VC34,HC01_MOE_VC34,HC02_EST_VC34,HC02_MOE_VC34,HC03_EST_VC34,HC03_MOE_VC34,HC04_EST_VC34,HC04_MOE_VC34,HC01_EST_VC35,HC01_MOE_VC35,HC02_EST_VC35,HC02_MOE_VC35,HC03_EST_VC35,HC03_MOE_VC35,HC04_EST_VC35,HC04_MOE_VC35,HC01_EST_VC38,HC01_MOE_VC38,HC02_EST_VC38,HC02_MOE_VC38,HC03_EST_VC38,HC03_MOE_VC38,HC04_EST_VC38,HC04_MOE_VC38,HC01_EST_VC39,HC01_MOE_VC39,HC02_EST_VC39,HC02_MOE_VC39,HC03_EST_VC39,HC03_MOE_VC39,HC04_EST_VC39,HC04_MOE_VC39,HC01_EST_VC40,HC01_MOE_VC40,HC02_EST_VC40,HC02_MOE_VC40,HC03_EST_VC40,HC03_MOE_VC40,HC04_EST_VC40,HC04_MOE_VC40,HC01_EST_VC41,HC01_MOE_VC41,HC02_EST_VC41,HC02_MOE_VC41,HC03_EST_VC41,HC03_MOE_VC41,HC04_EST_VC41,HC04_MOE_VC41,HC01_EST_VC42,HC01_MOE_VC42,HC02_EST_VC42,HC02_MOE_VC42,HC03_EST_VC42,HC03_MOE_VC42,HC04_EST_VC42,HC04_MOE_VC42,HC01_EST_VC45,HC01_MOE_VC45,HC02_EST_VC45,HC02_MOE_VC45,HC03_EST_VC45,HC03_MOE_VC45,HC04_EST_VC45,HC04_MOE_VC45,HC01_EST_VC46,HC01_MOE_VC46,HC02_EST_VC46,HC02_MOE_VC46,HC03_EST_VC46,HC03_MOE_VC46,HC04_EST_VC46,HC04_MOE_VC46,HC01_EST_VC47,HC01_MOE_VC47,HC02_EST_VC47,HC02_MOE_VC47,HC03_EST_VC47,HC03_MOE_VC47,HC04_EST_VC47,HC04_MOE_VC47,HC01_EST_VC48,HC01_MOE_VC48,HC02_EST_VC48,HC02_MOE_VC48,HC03_EST_VC48,HC03_MOE_VC48,HC04_EST_VC48,HC04_MOE_VC48,HC01_EST_VC49,HC01_MOE_VC49,HC02_EST_VC49,HC02

In [4]:
#drop second headers
acs_16 = acs_16.drop(acs_16.index[0])
#only keep columns that look at total categories
acs_16 = acs_16.loc[:, acs_16.columns.str.startswith('HC01') | acs_16.columns.str.startswith('GEO')]

In [5]:
#change all datatypes except geometry titles to float
cols_16 = acs_16.columns.drop('GEO.id').drop('GEO.id2').drop('GEO.display-label')
acs_16[cols_16] = acs_16[cols_16].apply(pd.to_numeric, errors='coerce', downcast='float')

#add census tract id column from GEO.id2
acs_16['ct2010'] = acs_16['GEO.id2'].str[-6:]

In [6]:
#import census tract shapefile
file = 'census_tracts_2010/geo_export_cc241bec-2a20-4016-b701-bff63fa878ae.shp'
ct = gpd.read_file(file)

In [7]:
#merge acs df with census tracts
acs_ct_16 = pd.merge(ct, acs_16, how='left', on=['ct2010', 'ct2010'])

In [8]:
acs_ct_16.head()

,boro_code,boro_ct201,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng,geometry,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC01_EST_VC03,HC01_MOE_VC03,HC01_EST_VC04,HC01_MOE_VC04,HC01_EST_VC05,HC01_MOE_VC05,HC01_EST_VC06,HC01_MOE_VC06,HC01_EST_VC07,HC01_MOE_VC07,HC01_EST_VC08,HC01_MOE_VC08,HC01_EST_VC09,HC01_MOE_VC09,HC01_EST_VC10,HC01_MOE_VC10,HC01_EST_VC12,HC01_MOE_VC12,HC01_EST_VC15,HC01_MOE_VC15,HC01_EST_VC16,HC01_MOE_VC16,HC01_EST_VC19,HC01_MOE_VC19,HC01_EST_VC20,HC01_MOE_VC20,HC01_EST_VC21,HC01_MOE_VC21,HC01_EST_VC22,HC01_MOE_VC22,HC01_EST_VC23,HC01_MOE_VC23,HC01_EST_VC24,HC01_MOE_VC24,HC01_EST_VC25,HC01_MOE_VC25,HC01_EST_VC26,HC01_MOE_VC26,HC01_EST_VC28,HC01_MOE_VC28,HC01_EST_VC29,HC01_MOE_VC29,HC01_EST_VC32,HC01_MOE_VC32,HC01_EST_VC33,HC01_MOE_VC33,HC01_EST_VC34,HC01_MOE_VC34,HC01_EST_VC35,HC01_MOE_VC35,HC01_EST_VC38,HC01_MOE_VC38,HC01_EST_VC39,HC01_MOE_VC39,HC01_EST_VC40,HC01_MOE_VC40,HC01_EST_VC41,HC01_MOE_VC41,HC01_EST_VC42,HC01_MOE_VC42,HC01_EST_VC45,HC01_MOE_VC45,HC01_EST_VC46,HC01_MOE_VC46,HC01_EST_VC47,HC01_MOE_VC47,HC01_EST_VC48,HC01_MOE_VC48,HC01_EST_VC49,HC01_MOE_VC49,HC01_EST_VC50,HC01_MOE_VC50,HC01_EST_VC53,HC01_MOE_VC53,HC01_EST_VC54,HC01_MOE_VC54,HC01_EST_VC55,HC01_MOE_VC55,HC01_EST_VC56,HC01_MOE_VC56,HC01_EST_VC57,HC01_MOE_VC57,HC01_EST_VC58,HC01_MOE_VC58,HC01_EST_VC59,HC01_MOE_VC59,HC01_EST_VC60,HC01_MOE_VC60,HC01_EST_VC61,HC01_MOE_VC61,HC01_EST_VC63,HC01_MOE_VC63,HC01_EST_VC66,HC01_MOE_VC66,HC01_EST_VC67,HC01_MOE_VC67,HC01_EST_VC68,HC01_MOE_VC68,HC01_EST_VC69,HC01_MOE_VC69,HC01_EST_VC72,HC01_MOE_VC72,HC01_EST_VC73,HC01_MOE_VC73
0,5,5000900,Staten Island,I,000900,9,SI22,West New Brighton-New Brighton-St. George,3903,2.497010e+06,7729.016794,POLYGON ((-74.07920577013245 40.64343078374567...,1400000US36047000900,36047000900,"Census Tract 9, Kings County, New York",4245.0,369.0,6.9,2.6,5.9,1.3,8.4,2.5,55.500000,4.7,9.2,2.8,7.2,2.5,2.0,1.0,4.8,2.1,33.400002,0.9,52.200001,3.4,47.799999,3.4,93.400002,2.5,71.300003,5.2,6.500000,2.7,0.0,0.7,14.3,3.9,0.0,0.7,1.300000,1.1,6.6,2.5,6.400000,2.7,68.199997,5.1,3951.0,345.0,19.000000,4.6,15.100000,4.0,3.900000,2.0,3725.0,318.0,56.000000,6.4,36.599998,6.3,4.3,2.1,3.1,1.9,3345.0,299.0,3.500000,2.0,5.3,1.8,5.800000,2.6,41.900002,5.3,43.599998,5.4,3725.0,318.0,11.7,2.6,3.5,2.2,3.8,1.9,4.5,1.9,7.2,2.6,10.1,3.1,3.4,1.7,47.000000,4.7,81500.0,13615.0,3830.0,364.0,6.900000,2.6,2.1,1.2,91.000000,3.0,3.7,NaN,7.8,NaN
1,5,5000900,Staten Island,I,000900,9,SI22,West New Brighton-New Brighton-St. George,3903,2.497010e+06,7729.016794,POLYGON ((-74.07920577013245 40.64343078374567...,1400000US36061000900,36061000900,"Census Tract 9, New York County, New York",1733.0,169.0,7.6,2.8,5.8,2.3,5.0,2.6,60.200001,5.3,12.2,4.0,7.3,3.3,1.2,0.5,0.7,0.6,33.700001,1.1,57.500000,4.3,42.500000,4.3,95.800003,2.7,74.599998,6.8,2.900000,2.2,0.0,1.7,12.8,5.0,0.3,0.5,5.300000,4.8,4.2,2.7,9.200000,4.8,70.199997,6.7,1602.0,150.0,24.299999,6.5,20.600000,6.1,3.700000,2.1,1525.0,141.0,44.700001,7.3,48.000000,7.4,6.0,2.6,1.2,1.4,1414.0,149.0,1.100000,1.2,4.0,2.3,10.900000,5.5,40.700001,6.1,43.400002,5.7,1525.0,141.0,3.1,1.5,3.9,2.3,4.7,2.3,1.8,1.8,8.1,3.5,9.5,2.9,2.8,1.4,58.200001,4.6,99286.0,7268.0,1733.0,169.0,4.300000,2.2,1.3,1.0,94.500000,2.1,4.3,NaN,7.4,NaN
2,1,1009800,Manhattan,I,009800,98,MN19,Turtle Bay-East Midtown,3808,1.906016e+06,5534.199811,POLYGON ((-73.96432543478758 40.75638153099091...,1400000US36005009800,36005009800,"Census Tract 98, Bronx County, New York",5306.0,476.0,5.5,2.0,14.7,3.8,13.6,3.0,22.500000,3.6,16.0,2.8,11.8,3.4,8.4,2.8,7.4,2.6,39.900002,6.4,44.599998,3.0,55.400002,3.0,97.199997,2.8,41.400002,8.1,29.200001,6.6,0.0,0.6,3.4,2.8,0.0,0.6,23.299999,7.2,2.8,2.8,71.300003,6.2,2.600000,1.6,5014.0,449.0,56.500000,7.2,37.400002,7.2,19.200001,6.6,4423.0,394.0,46.400002,5.1,33.400002,5.6,14.7,3.5,5.5,2.5,3511.0,319.0,18.000000,4.5,28.9,5.6,25.700001,5.0,17.799999,5.2,9.600000,4.4,4423.0,394.0,19.5,6.6,6.6,2.8,13.3,4.6,8.1,3.0,11.0,3.5,7.1,2.5

In [9]:
#get race descriptors from the acs

#HC01_EST_VC19 = total
#HC01_EST_VC20 = white
#HC01_EST_VC21 = black
#HC01_EST_VC22 = american indian
#HC01_EST_VC23 = asian
#HC01_EST_VC24 = native hawaiian
#HC01_EST_VC25 = other
#HC01_EST_VC26 = two or more races

#HC01_EST_VC28 = hispanic or latino (proportion of total)
#HC01_EST_VC29 = non-hispanic white 

race_16 = acs_ct_16[['geometry', 'GEO.id2', 'HC01_EST_VC01', 'HC01_EST_VC20', 'HC01_EST_VC21', 'HC01_EST_VC22', 
                     'HC01_EST_VC23', 'HC01_EST_VC24', 'HC01_EST_VC25', 'HC01_EST_VC26', 'ct2010', 'ntacode']]

In [10]:
race_16.head()

,geometry,GEO.id2,HC01_EST_VC01,HC01_EST_VC20,HC01_EST_VC21,HC01_EST_VC22,HC01_EST_VC23,HC01_EST_VC24,HC01_EST_VC25,HC01_EST_VC26,ct2010,ntacode
0,POLYGON ((-74.07920577013245 40.64343078374567...,36047000900,4245.0,71.300003,6.500000,0.0,14.3,0.0,1.300000,6.6,000900,SI22
1,POLYGON ((-74.07920577013245 40.64343078374567...,36061000900,1733.0,74.599998,2.900000,0.0,12.8,0.3,5.300000,4.2,000900,SI22
2,POLYGON ((-73.96432543478758 40.75638153099091...,36005009800,5306.0,41.400002,29.200001,0.0,3.4,0.0,23.299999,2.8,009800,MN19
3,POLYGON ((-73.96432543478758 40.75638153099091...,36047009800,8144.0,16.200001,3.700000,0.0,45.5,0.0,33.099998,1.4,009800,MN19
4,POLYGON ((-73.96432543478758 40.75638153099091...,36061009800,7284.0,80.599998,2.800000,0.0,14.4,0.0,1.200000,1.0,009800,MN19


In [11]:
#create race distribution dataframe with total number of people in each classification
race = race_16[['geometry', 'GEO.id2', 'HC01_EST_VC01']]
race = race.rename(columns={'HC01_EST_VC01':'popn_total'})
race['popn_white'] = race['popn_total'] * race_16['HC01_EST_VC20']/100
race['popn_black'] = race['popn_total'] * race_16['HC01_EST_VC21']/100
race['popn_amindian'] = race['popn_total'] * race_16['HC01_EST_VC22']/100
race['popn_asian'] = race['popn_total'] * race_16['HC01_EST_VC23']/100
race['popn_nhawaii'] = race['popn_total'] * race_16['HC01_EST_VC24']/100
race['popn_other'] = race['popn_total'] * race_16['HC01_EST_VC25']/100
race['popn_twomore'] = race['popn_total'] * race_16['HC01_EST_VC26']/100
race['ct2010'] = race_16['ct2010']
race['ntacode'] = race_16['ntacode']
race.head()

,geometry,GEO.id2,popn_total,popn_white,popn_black,popn_amindian,popn_asian,popn_nhawaii,popn_other,popn_twomore,ct2010,ntacode
0,POLYGON ((-74.07920577013245 40.64343078374567...,36047000900,4245.0,3026.685059,275.924988,0.0,607.034973,0.000,55.185001,280.170013,000900,SI22
1,POLYGON ((-74.07920577013245 40.64343078374567...,36061000900,1733.0,1292.817993,50.257004,0.0,221.824005,5.199,91.849007,72.785995,000900,SI22
2,POLYGON ((-73.96432543478758 40.75638153099091...,36005009800,5306.0,2196.684082,1549.352051,0.0,180.404007,0.000,1236.297974,148.567993,009800,MN19
3,POLYGON ((-73.96432543478758 40.75638153099091...,36047009800,8144.0,1319.328125,301.328003,0.0,3705.520020,0.000,2695.663818,114.015999,009800,MN19
4,POLYGON ((-73.96432543478758 40.75638153099091...,36061009800,7284.0,5870.903809,203.951996,0.0,1048.895996,0.000,87.408005,72.839996,009800,MN19


In [82]:
race.to_csv('race_distribution.csv')

In [12]:
#get income in the last 12 months from the acs

#HC01_EST_VC53 = total population over 15
#HC01_EST_VC54 = $1 - $9,999
#HC01_EST_VC55 = $10,000 - $14,999
#HC01_EST_VC56 = $15,000 - $24,999
#HC01_EST_VC57 = $25,000 - $34,999
#HC01_EST_VC58 = $35,000 - $49,999
#HC01_EST_VC59 = $50,000 - $64,999
#HC01_EST_VC60 = $65,000 - $74,999
#HC01_EST_VC61 = $75,000 +
#HC01_EST_VC63 = median income

income_16 = acs_ct_16[['geometry','GEO.id2', 'HC01_EST_VC53', 'HC01_EST_VC54', 'HC01_EST_VC55', 'HC01_EST_VC56', 
                       'HC01_EST_VC57', 'HC01_EST_VC58', 'HC01_EST_VC59', 'HC01_EST_VC60', 'HC01_EST_VC61', 
                       'ct2010', 'ntacode']]
income_16.head()


,geometry,GEO.id2,HC01_EST_VC53,HC01_EST_VC54,HC01_EST_VC55,HC01_EST_VC56,HC01_EST_VC57,HC01_EST_VC58,HC01_EST_VC59,HC01_EST_VC60,HC01_EST_VC61,ct2010,ntacode
0,POLYGON ((-74.07920577013245 40.64343078374567...,36047000900,3725.0,11.7,3.5,3.8,4.5,7.2,10.1,3.4,47.000000,000900,SI22
1,POLYGON ((-74.07920577013245 40.64343078374567...,36061000900,1525.0,3.1,3.9,4.7,1.8,8.1,9.5,2.8,58.200001,000900,SI22
2,POLYGON ((-73.96432543478758 40.75638153099091...,36005009800,4423.0,19.5,6.6,13.3,8.1,11.0,7.1,3.9,10.400000,009800,MN19
3,POLYGON ((-73.96432543478758 40.75638153099091...,36047009800,5781.0,19.0,12.5,19.0,7.7,6.7,1.7,1.1,2.900000,009800,MN19
4,POLYGON ((-73.96432543478758 40.75638153099091...,36061009800,6880.0,5.3,2.9,4.4,6.3,8.8,5.7,7.6,54.500000,009800,MN19


In [13]:
#create income distribution dataframe with proportions and column names

income = income_16[['geometry', 'GEO.id2', 'HC01_EST_VC53']]
income = income.rename(columns={'HC01_EST_VC53':'popn_abov15'})

income['from1'] = income['popn_abov15'] * income_16['HC01_EST_VC54']/100
income['from10'] = income['popn_abov15'] * income_16['HC01_EST_VC55']/100
income['from15'] = income['popn_abov15'] * income_16['HC01_EST_VC56']/100
income['from25'] = income['popn_abov15'] * income_16['HC01_EST_VC57']/100
income['from35'] = income['popn_abov15'] * income_16['HC01_EST_VC58']/100
income['from50'] = income['popn_abov15'] * income_16['HC01_EST_VC59']/100
income['from65'] = income['popn_abov15'] * income_16['HC01_EST_VC60']/100
income['from75'] = income['popn_abov15'] * income_16['HC01_EST_VC61']/100
income['none'] = income['popn_abov15'] - (income['from1']+income['from10']+income['from15']+income['from25']
                            +income['from35']+income['from50']+income['from65']+income['from75'])

income['ct2010'] = income_16['ct2010']
income['ntacode'] = income_16['ntacode']
income.head()

,geometry,GEO.id2,popn_abov15,from1,from10,from15,from25,from35,from50,from65,from75,none,ct2010,ntacode
0,POLYGON ((-74.07920577013245 40.64343078374567...,36047000900,3725.0,435.825012,130.375000,141.550003,167.625000,268.200012,376.225006,126.650002,1750.750000,327.800049,000900,SI22
1,POLYGON ((-74.07920577013245 40.64343078374567...,36061000900,1525.0,47.275002,59.474998,71.674995,27.450001,123.525009,144.875000,42.700001,887.549988,120.475098,000900,SI22
2,POLYGON ((-73.96432543478758 40.75638153099091...,36005009800,4423.0,862.484985,291.917999,588.259033,358.263000,486.529999,314.032990,172.497009,459.992004,889.022949,009800,MN19
3,POLYGON ((-73.96432543478758 40.75638153099091...,36047009800,5781.0,1098.390015,722.625000,1098.390015,445.136993,387.326996,98.277000,63.591000,167.649002,1699.614014,009800,MN19
4,POLYGON ((-73.96432543478758 40.75638153099091...,36061009800,6880.0,364.640015,199.520004,302.720001,433.440002,605.440002,392.160004,522.880005,3749.600098,309.600098,009800,MN19


In [14]:
income.to_csv('income_distribution.csv')

In [31]:
#  To map the race race and income distribution of people that have access to citi bikes docks within a 5-minute walk,
#I conducted some analysis using sql and network analysis in carto.
#  I first used carto's distance analysis on all citi bike station points to get the polygons of how far people can
#walk within 5 minutes from citi bike stations. This shapefile is called 'citibike_station_walkshed_disjointed'.
#  I then joined all the polygons ('citibike_station_walkshed').
#  In carto, I used 'citibike_station_walkshed' and 'race_distribution.csv' to get the percentage area of the 
#census tracts that were covered by the walkshed and used the percentage covered area to get the percentage
#population of people in each census tract that overlaps with the walkshed. 
#  This analysis gave me the estimated absolute value of people in each census tract that can access citi bike 
#docks within a 5-minute walk.
#  I did the same process with 'income_distribution.csv' and imported it back to the notebook to get the
#total walkshed distrubtion for both race and income

In [18]:
race_dist = pd.read_csv('citibike_walkshed_race_distribution.csv', na_values=['(NA)']).fillna(0)
income_dist = pd.read_csv('citibike_walkshed_income_distribution.csv', na_values=['(NA)']).fillna(0)

In [19]:
#change order and datatypes
race_dist = race_dist[['popn_total', 'percentcovered', 'total_popinwalkshed', 'white_popinwalkshed', 
                       'black_popinwalkshed', 'amindian_popinwalkshed', 'asian_popinwalkshed', 
                       'nhawaii_popinwalkshed','other_popinwalkshed', 'twomore_popinwalkshed']] 

race_dist['popn_total'] = pd.to_numeric(race_dist['popn_total'])
race_dist['percentcovered'] = pd.to_numeric(race_dist['percentcovered'])
race_dist['total_popinwalkshed'] = pd.to_numeric(race_dist['total_popinwalkshed'])
race_dist['white_popinwalkshed'] = pd.to_numeric(race_dist['white_popinwalkshed'])
race_dist['black_popinwalkshed'] = pd.to_numeric(race_dist['black_popinwalkshed'])
race_dist['amindian_popinwalkshed'] = pd.to_numeric(race_dist['amindian_popinwalkshed'])
race_dist['asian_popinwalkshed'] = pd.to_numeric(race_dist['asian_popinwalkshed'])
race_dist['nhawaii_popinwalkshed'] = pd.to_numeric(race_dist['nhawaii_popinwalkshed'])
race_dist['other_popinwalkshed'] = pd.to_numeric(race_dist['other_popinwalkshed'])
race_dist['twomore_popinwalkshed'] = pd.to_numeric(race_dist['twomore_popinwalkshed'])

In [21]:
#get only census tracts that overlap with the walkshed
race_dist = race_dist[race_dist['percentcovered'] > 0]
race_dist.head()

,popn_total,percentcovered,total_popinwalkshed,white_popinwalkshed,black_popinwalkshed,amindian_popinwalkshed,asian_popinwalkshed,nhawaii_popinwalkshed,other_popinwalkshed,twomore_popinwalkshed
8,2073,0.062556,129.679347,46.814242,53.168532,2.593587,1.815511,1.037435,17.636391,6.613647
11,4488,0.888436,3987.300193,3644.392377,127.593615,3.987301,0.000000,7.974601,167.466608,31.898404
13,1801,0.971634,1749.912039,1093.695024,85.745690,0.000000,162.741829,0.000000,293.985223,115.494195
23,4224,0.424486,1793.030199,1282.016592,170.337869,0.000000,164.958778,0.000000,98.616661,77.100307
24,3916,0.999964,3915.858615,3011.295275,46.990303,19.579293,595.210510,0.000000,74.401314,168.381920


In [24]:
#get race distribution of citibike walkshed
race_dist['white_per'] = race_dist['white_popinwalkshed'].sum()/race_dist['total_popinwalkshed'].sum()
race_dist['black_per'] = race_dist['black_popinwalkshed'].sum()/race_dist['total_popinwalkshed'].sum()
race_dist['amindian_per'] = race_dist['amindian_popinwalkshed'].sum()/race_dist['total_popinwalkshed'].sum()
race_dist['asian_per'] = race_dist['asian_popinwalkshed'].sum()/race_dist['total_popinwalkshed'].sum()
race_dist['nhawaii_per'] = race_dist['nhawaii_popinwalkshed'].sum()/race_dist['total_popinwalkshed'].sum()
race_dist['other_per'] = race_dist['other_popinwalkshed'].sum()/race_dist['total_popinwalkshed'].sum()
race_dist['twomore_per'] = race_dist['twomore_popinwalkshed'].sum()/race_dist['total_popinwalkshed'].sum()

citi_race_dist = race_dist[['white_per', 'black_per', 'amindian_per', 'asian_per', 'nhawaii_per', 'other_per', 'twomore_per']]
citi_race_dist.mean()

white_per       0.519586
black_per       0.154774
amindian_per    0.004125
asian_per       0.124022
nhawaii_per     0.000472
other_per       0.159613
twomore_per     0.037397
dtype: float64

In [25]:
#get race distribution of nyc (all boroughs)
race['white_per'] = race['popn_white'].sum()/race['popn_total'].sum()
race['black_per'] = race['popn_black'].sum()/race['popn_total'].sum()
race['amindian_per'] = race['popn_amindian'].sum()/race['popn_total'].sum()
race['asian_per'] = race['popn_asian'].sum()/race['popn_total'].sum()
race['nhawaii_per'] = race['popn_nhawaii'].sum()/race['popn_total'].sum()
race['other_per'] = race['popn_other'].sum()/race['popn_total'].sum()
race['twomore_per'] = race['popn_twomore'].sum()/race['popn_total'].sum()

nyc_race_dist = race[['white_per', 'black_per', 'amindian_per', 'asian_per', 'nhawaii_per', 'other_per', 'twomore_per']]
nyc_race_dist.mean()

white_per       0.426301
black_per       0.229309
amindian_per    0.004237
asian_per       0.131956
nhawaii_per     0.000486
other_per       0.172306
twomore_per     0.035379
dtype: float32

In [26]:
#change order and datatypes
income_dist['popn_abov15'] = pd.to_numeric(income_dist['popn_abov15'])
income_dist['percentcovered'] = pd.to_numeric(income_dist['percentcovered'])
income_dist['popn15_inwalkshed'] = pd.to_numeric(income_dist['total_pop15_inwalkshed'])
income_dist['from1'] = pd.to_numeric(income_dist['from1_popinwalkshed'])
income_dist['from10'] = pd.to_numeric(income_dist['from10_popinwalkshed'])
income_dist['from15'] = pd.to_numeric(income_dist['from15_popinwalkshed'])
income_dist['from25'] = pd.to_numeric(income_dist['from25_popinwalkshed'])
income_dist['from35'] = pd.to_numeric(income_dist['from35_popinwalkshed'])
income_dist['from50'] = pd.to_numeric(income_dist['from50_popinwalkshed'])
income_dist['from65'] = pd.to_numeric(income_dist['from65_popinwalkshed'])
income_dist['from75'] = pd.to_numeric(income_dist['from75_popinwalkshed'])
income_dist['none'] = pd.to_numeric(income_dist['none_popinwalkshed'])

income_dist = income_dist[['popn_abov15', 'percentcovered', 'popn15_inwalkshed', 'from1', 
                           'from10', 'from15', 'from25',
                          'from35', 'from50', 'from65', 
                           'from75', 'none']] 


income_dist.head()

,popn_abov15,percentcovered,popn15_inwalkshed,from1,from10,from15,from25,from35,from50,from65,from75,none
0,5640,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.000,0.000,0.000,0.0000,0.00000
1,2126,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.000,0.000,0.000,0.0000,0.00000
2,3702,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.000,0.000,0.000,0.0000,0.00000
3,2130,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.000,0.000,0.000,0.0000,0.00000
4,7526,1.0,7526.0,760.12604,142.99399,398.87802,285.98798,835.386,699.918,353.722,3597.4277,451.56055


In [27]:
#filter by census tracts that overlap with the walkshed
income_dist = income_dist[income_dist['percentcovered'] > 0]
income_dist.head()

,popn_abov15,percentcovered,popn15_inwalkshed,from1,from10,from15,from25,from35,from50,from65,from75,none
4,7526,1.000000,7526.000000,760.126040,142.993990,398.878020,285.987980,835.386000,699.918000,353.722000,3597.427700,451.560550
5,4320,0.979108,4229.744591,587.934498,410.285225,545.637052,494.880117,621.772455,317.230844,42.297446,477.961139,731.745912
25,812,0.990690,804.440410,37.808697,42.635342,34.590938,61.137469,64.355233,90.901770,24.133212,389.349159,59.528603
27,3827,0.424486,1624.509131,191.692090,108.842112,99.095053,125.087203,164.075439,71.478402,95.846043,550.708595,217.684266
29,6329,0.999961,6328.753244,443.012727,202.520124,316.437662,398.711474,417.697714,601.231558,120.246312,3417.526752,411.368731


In [28]:
#get income distribution of citibike walkshed
income_dist['from1_per'] = income_dist['from1'].sum()/income_dist['popn15_inwalkshed'].sum()
income_dist['from10_per'] = income_dist['from10'].sum()/income_dist['popn15_inwalkshed'].sum()
income_dist['from15_per'] = income_dist['from15'].sum()/income_dist['popn15_inwalkshed'].sum()
income_dist['from25_per'] = income_dist['from25'].sum()/income_dist['popn15_inwalkshed'].sum()
income_dist['from35_per'] = income_dist['from35'].sum()/income_dist['popn15_inwalkshed'].sum()
income_dist['from50_per'] = income_dist['from50'].sum()/income_dist['popn15_inwalkshed'].sum()
income_dist['from65_per'] = income_dist['from65'].sum()/income_dist['popn15_inwalkshed'].sum()
income_dist['from75_per'] = income_dist['from75'].sum()/income_dist['popn15_inwalkshed'].sum()
income_dist['none_per'] = income_dist['none'].sum()/income_dist['popn15_inwalkshed'].sum()

citi_income_dist = income_dist[['from1_per', 'from10_per', 'from15_per', 'from25_per', 'from35_per', 'from50_per', 
                               'from65_per', 'from75_per', 'none_per', ]]
citi_income_dist.mean()

from1_per     0.151000
from10_per    0.073421
from15_per    0.105831
from25_per    0.079195
from35_per    0.092376
from50_per    0.080137
from65_per    0.038543
from75_per    0.227761
none_per      0.151736
dtype: float64

In [30]:
#get income distribution of nyc (all boroughs)
income['from1_per'] = income['from1'].sum()/income['popn_abov15'].sum()
income['from10_per'] = income['from10'].sum()/income['popn_abov15'].sum()
income['from15_per'] = income['from15'].sum()/income['popn_abov15'].sum()
income['from25_per'] = income['from25'].sum()/income['popn_abov15'].sum()
income['from35_per'] = income['from35'].sum()/income['popn_abov15'].sum()
income['from50_per'] = income['from50'].sum()/income['popn_abov15'].sum()
income['from65_per'] = income['from65'].sum()/income['popn_abov15'].sum()
income['from75_per'] = income['from75'].sum()/income['popn_abov15'].sum()
income['none_per'] = income['none'].sum()/income['popn_abov15'].sum()

nyc_income_dist = income[['from1_per', 'from10_per', 'from15_per', 'from25_per', 'from35_per', 'from50_per', 
                               'from65_per', 'from75_per', 'none_per', ]]

nyc_income_dist.mean()

from1_per     0.166325
from10_per    0.081973
from15_per    0.121794
from25_per    0.088165
from35_per    0.098145
from50_per    0.077038
from65_per    0.034239
from75_per    0.157716
none_per      0.174602
dtype: float32